In [ ]:
pip install gemini

  Using cached gemini-0.30.2.tar.gz (1.6 MB)
  Preparing metadata (setup.py) ... done
  Using cached inheritance-0.1.5.tar.gz (27 kB)
  Preparing metadata (setup.py) ... done
  Using cached geneimpacts-0.3.7.tar.gz (40 kB)
  Preparing metadata (setup.py) ... done
  Using cached pysam-0.23.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached cyvcf2-0.31.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.5 kB)
  Using cached pybedtools-0.12.0.tar.gz (12.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached bottle-0.13.3-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached ipython-cluster-helper-0.6.4.tar.gz (22 kB)
  Preparing metadata (setup.py) ... done
  Using cached bx_python-0.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.1 kB)
  Using cached openpyxl-1.8.6-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached 

In [ ]:
# code source: https://milumon.medium.com/creating-custom-chatbots-using-csv-data-with-python-and-openai-api-0486d8992b24
import google.generativeai as genai
from google.colab import userdata

GEMINI_API_KEY = userdata.get('key')
genai.configure(api_key=GEMINI_API_KEY)

In [ ]:
with open("SeafoodWatch_data_full.csv") as file:
    product_data = file.read()

In [ ]:
context = []

rules = """
You are a seafood sustainability expert from Seafood Watch and you have to recommend seafood based on environmental impact and tell consumers whether they need to buy or avoid the seafood type and reason why.\
Be cordial at all times. \
If consumers haven't chosen yet, recommend alternatives or certified seafood type. \
Offer recommendations if consumers are undecided between the seafood type available. \
Based on the consumer's preference or past questions, you may suggest a specific seafood type they might consider environmentally sustainable. \
After selecting a seafood type, confirm whether they should buy or avoid the seafood type, and ask if they have any further questions. \
When decided whether to buy the seafood type, provide a summary of the conversation, including the seafood type, recommendation, origin, fishing gear, and farming method. \
If the seafood type and/or location is not in the list, say that it is not in the Seafood Watch database that you know. If the seafood type does exist in the list but in another location than the consumer has asked about, you can still talk about the list.
After saying something about the list, you may still recommend whether to buy or avoid the seafood type from the place that the consumer asked about.
If user ask you for recipe for a certain seafood, find some recipe online if you don't know. \
Below is your environmentally sustainable seafood type recommendation with a reason based on the information discussed. \
"""

context.append({'role': 'system', 'content': f"""{rules} {product_data}"""})

In [ ]:
def fetch_messages(messages, model="gemini-1.5-pro-002", temperature=0):
    formatted_prompt = "\n".join([f"{msg['role'].capitalize()}: {msg['content']}" for msg in messages])
    gemini_model = genai.GenerativeModel(model)
    response = gemini_model.generate_content(formatted_prompt, generation_config={"temperature": temperature})

    return response.text

In [ ]:
def refresh_conversation(chat):
    context.append({'role': 'user', 'content': f"{chat}"})
    response = fetch_messages(context, temperature=0.7)
    context.append({'role': 'assistant', 'content': f"{response}"})
    print(response)

In [ ]:
def main():

    while True:
        message = input("Hello! How can I help? (type 'exit' to leave): ")

        if message.lower() == 'exit':
            break

        refresh_conversation(message)

if __name__ == '__main__':
    main()

Hello! How can I help? (type 'exit' to leave): What if my grandma is coming over and she cannot easily chew or cut fish?


KeyboardInterrupt: 

Prompts tested:
- What if im allergic to shellfish?
- Are farmed fish always less sustainable?
- What are the fish with less bones?
- Isn't eel a type of seafood?
- What are the best type of seafood for sushi?
- What do you recommend if I'm vegan?


In [ ]:
import pandas as pd
df = pd.read_csv("SeafoodWatch_data_full.csv")

df = df[['species', 'origin', 'recommendation']].dropna()

recommendation_map = {
    "BEST CHOICE": 2,
    "GOOD ALTERNATIVE": 1,
    "CERTIFIED": 1.5,
    "AVOID": 0
}

df['recommendation_score'] = df['recommendation'].map(recommendation_map)

df.head()


,species,origin,recommendation,recommendation_score
0,Abalone,Worldwide,BEST CHOICE,2.0
1,Abalone,Worldwide,BEST CHOICE,2.0
2,Abalone,Australia,CERTIFIED,1.5
3,Almaco jack,Worldwide,BEST CHOICE,2.0
4,Almaco jack,Mexico,GOOD ALTERNATIVE,1.0


In [ ]:
import plotly.express as px

fig = px.scatter(
    df,
    x="species",
    y="origin",
    color="recommendation",
    size="recommendation_score",
    title="Seafood Type vs Origin Clustering by Recommendation",
    hover_data=["species", "origin", "recommendation"]
)

fig.show()


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
import plotly.express as px
import time

df = pd.read_csv("SeafoodWatch_data_full.csv")
df = df[['species', 'origin', 'recommendation']].dropna()
df['constant_size'] = 10

recommendation_map = {
    "BEST CHOICE": 1.5,
    "GOOD ALTERNATIVE": 1.5,
    "CERTIFIED": 1.5,
    "AVOID": 1.5
}

color_mapping = {
    "BEST CHOICE": "green",
    "GOOD ALTERNATIVE": "yellow",
    "CERTIFIED": "blue",
    "AVOID": "red"
}


rec_colors =  {'green', 'yellow', 'blue', 'red'}

df['recommendation_score'] = df['recommendation'].map(recommendation_map)

geolocator = Nominatim(user_agent="seafood_locator")

def get_lat_lon(location):
    try:
        loc = geolocator.geocode(location)
        if loc:
            return pd.Series([loc.latitude, loc.longitude])
    except:
        return pd.Series([None, None])
    return pd.Series([None, None])

unique_origins = df['origin'].dropna().unique()
location_df = pd.DataFrame(unique_origins, columns=['origin'])
location_df[['lat', 'lon']] = location_df['origin'].apply(get_lat_lon)

time.sleep(1)

df = df.merge(location_df, on='origin', how='left')

df = df.dropna(subset=['lat', 'lon'])

fig = px.scatter_geo(
    df,
    lat='lat',
    lon='lon',
    color='recommendation',
    hover_name='species',
    title="Global Map of Seafood Types by Origin and Recommendation",
    projection="natural earth",
    color_discrete_map=color_mapping,
)

fig.update_traces(marker=dict(size=10))

fig.show()